In [ ]:
import pandas as pd
import cv2 as cv
import numpy as np
import os

In [4]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
df = pd.read_csv('train_boundingboxes.csv',sep=', ')

In [ ]:
imglist = sorted_alphanumeric(os.listdir('Train'))
widthList = []
heightList = []
for img in imglist:
    image = cv.imread('Train/'+img)
    height, width, channels = image.shape
    widthList.insert(len(widthList),width)
    heightList.insert(len(heightList),height)

In [ ]:
def findX(xmin,xmax,width):
    return (xmin + xmax)/ 2.0 / width

def findY(ymin,ymax,height):
    return (ymin + ymax)/ 2.0 / height

def findW(xmin,xmax,width):
    return abs(xmin - xmax) / width

def findH(ymin,ymax,height):
    return abs(ymin - ymax) / height

df['pwidth'] = widthList
df['pheight'] = heightList
df['x_centre'] = df.apply(lambda x: findX(x['X_min'],x['X_max'],x['pwidth']),axis=1)
df['y_centre'] = df.apply(lambda x: findY(x['Y_min'],x['Y_max'],x['pheight']),axis=1)
df['width'] = df.apply(lambda x: findW(x['X_min'],x['X_max'],x['pwidth']),axis=1)
df['height'] = df.apply(lambda x: findH(x['Y_min'],x['Y_max'],x['pheight']),axis=1)
df['Class'] = 0

In [ ]:
dfresult = df[['Class','x_centre','y_centre','width','height']]
dfresult = dfresult.round(6)
dfresult = dfresult.astype(str)

In [ ]:
i=0
for row in dfresult.values:
    filename = 'data/obj/{}.txt'.format(i)
    row.tofile(filename, sep=" ", format="%s")
    i+=1

In [5]:
path = 'data/obj/'
imgList = sorted_alphanumeric(os.listdir('Train'))
textFile = open('data/train.txt','w')
testFile = open('data/test.txt','w')
# i = 0
# for img in imgList:
#     if(i < 9000):
#         imgPath = path+img+'\n'
#         textFile.write(imgPath)
#         i+=1
#     else:
#         imgPath = path+img+'\n'
#         testFile.write(imgPath)
#         i = 1000
for img in imgList:
    imgPath = path+img+'.jpg'+'\n'
    textFile.write(imgPath)

testFile = open('data/test2.txt','w')
imgList = sorted_alphanumeric(os.listdir('Train'))
path = 'data/Test/' 
i = 1000
for img in imgList:
    imgPath = path+str(i)+'.jpg'+'\n'
    i+=1
    testFile.write(imgPath)

NameError: name 'i' is not defined

In [ ]:
from rebox import BBox
from rebox.formats import yolo,pascal
def yolo_to_pascal_voc(x_center, y_center, w, h,  image_w, image_h):
    w = w * image_w
    h = h * image_h
    x1 = ((2 * x_center * image_w) - w)/2
    y1 = ((2 * y_center * image_h) - h)/2
    x2 = x1 + w
    y2 = y1 + h
    return [x1, y1, x2, y2]

In [ ]:
# convert ur json to yaml then to csv its retarded how easy to do that instead

imglist = sorted_alphanumeric(os.listdir('Test'))
widthList = []
heightList = []
for img in imglist:
    image = cv.imread('Test/'+img)
    height, width, channels = image.shape
    widthList.insert(len(widthList),width)
    heightList.insert(len(heightList),height)
    
cleanup = pd.read_csv('convertcsv.csv')

cleanedfilename = []

for i in cleanup['filename']:
    cleanedfilename.insert(len(cleanedfilename),i.split('/')[2])
cleanup['filename'] = cleanedfilename

x_min = []
y_min = []
x_max = []
y_max = []

i = 0
for index, row in cleanup.iterrows():
    if (~pd.isnull(row['objects/0/relative_coordinates/width'])):
        # pascalb = yolo_to_pascal_voc(row['objects/0/relative_coordinates/center_x'],row['objects/0/relative_coordinates/center_y'],row['objects/0/relative_coordinates/width'],row['objects/0/relative_coordinates/height'],
        #                              widthList[i],heightList[i])
        # x_min.insert(len(x_min),pascalb[0])
        # y_min.insert(len(y_min),pascalb[1])
        # x_max.insert(len(x_max),pascalb[2])
        # y_max.insert(len(y_max),pascalb[3])
        
        yolob = BBox([row['objects/0/relative_coordinates/center_x'],row['objects/0/relative_coordinates/center_y'],row['objects/0/relative_coordinates/width'],row['objects/0/relative_coordinates/height']],yolo)
        pascalb = yolob.as_format(pascal, widthList[i], heightList[i])
        x_min.insert(len(x_min),pascalb.x1)
        y_min.insert(len(y_min),pascalb.y1)
        x_max.insert(len(x_max),pascalb.x2)
        y_max.insert(len(y_max),pascalb.y2)
    else:    
        x_min.insert(len(x_min),0)
        y_min.insert(len(y_min),0)
        x_max.insert(len(x_max),0)
        y_max.insert(len(y_max),0)
    i+=1

In [ ]:
result = pd.DataFrame()
result[' X_min'] = x_min
result[' Y_min'] = y_min
result[' X_max'] = x_max
result[' Y_max'] = y_max
result[result < 0] = 0
result = result.replace(r'nan', np.nan, regex=True)
result = result.fillna(0)

In [ ]:
result[' X_min'].astype(int)
result[' Y_min'].astype(int)
result[' X_max'].astype(int)
result[' Y_max'].astype(int)
result['ImageId'] = cleanup['filename']
result = result[['ImageId',' X_min',' Y_min',' X_max',' Y_max']]
result.to_csv('EE4211_group1.csv',index=False)